<div>
    <table style="border:2px solid white;" cellspacing="0" cellpadding="0" border-collapse: collapse; border-spacing: 0;>
      <tr> 
        <th style="background-color:white"> <img src="../media/ccal-logo-D3.png" width=225 height=225></th>
        <th style="background-color:white"> <img src="../media/logoMoores.jpg" width=175 height=175></th>
        <th style="background-color:white"> <img src="../media/GP.png" width=200 height=200></th>
        <th style="background-color:white"> <img src="../media/UCSD_School_of_Medicine_logo.png" width=175 height=175></th> 
        <th style="background-color:white"> <img src="../media/Broad.png" width=130 height=130></th> 
      </tr>
    </table>
</div>

<hr style="border: none; border-bottom: 3px solid #88BBEE;">

# **Onco-*GPS* Methodology**
## **Chapter 1. Downloading Data**

<div>
    <img style="float: left" src="../media/authors.png" width=800 height=40>
</div>

Before executing the chapters that follow, some input datasets need to be downloaded to the data directory from their sources, and some need to be modified slightly. To do this, please follow the steps below.

Go to the [next chapter(2)](2 Generating Oncogenic Activation Signature.ipynb). Back to [introduction chapter (0)](0 Introduction and Overview.ipynb)

<hr style="border: none; border-bottom: 3px solid #88BBEE;">
### 1. Set up notebook and import Computational Cancer Analysis Library ([CCAL](https://github.com/KwatME/ccal))

Run the cell below to set up this notebook's environment.

In [2]:
from environment import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### 2. Download the Cancer Cell Line Encyclopedia (CCLE) datasets from the CCLE web site

Watch [this quick video](https://www.youtube.com/watch?v=iTFm3n_JRw8&feature=youtu.be) or follow these steps:

1. Go [here](https://portals.broadinstitute.org/ccle_legacy/toa/termsOfAccess/20/) to begin making an account with CCLE.
2. Click "Accept" and enter your account information
3. Go the the "Browse" tab and select "Data"
4. At the bottom of the page beneath "Publication specific data files" click"Show available data"
5. Click "CCLE_MUT_EXPR_RPPA_OncoGPS.zip" to download
6. Unzip "CCLE_MUT_EXPR_RPPA_OncoGPS.zip" and move the 4 files inside to the onco-gps-paper-analysis/data directory.

### 3. Download the gene dependencies (RNAi) dataset from the Achilles project web site

Watch [this quick video](https://www.youtube.com/watch?v=wj0cJC9-XYw&feature=youtu.be) or follow these steps:

1. Go [here](https://portals.broadinstitute.org/achilles/users/sign_up) to make an account with Project Achilles.
2. Confirm your email with the confirmation link in an email you'll receive from Project Achilles.
3. Go [here](https://portals.broadinstitute.org/achilles/datasets/15/download) and click the "ExpandedGeneZSolsCleaned.csv" link to download the gene dependencies dataset
4. Move the downloaded "ExpandedGeneZSolsCleaned.csv" file to the onco-gps-paper-analysis/data directory

Now, run the cell below to rename the dataset.

In [ ]:
df = pd.read_csv('../data/ExpandedGeneZSolsCleaned.csv', index_col=0)
ccal.write_gct(df, '../data/achilles__gene_x_ccle_cellline.gct')

### 4. Download the CTRP v2 datasets from the Broad Institute

1. Go here: ftp://caftpd.nci.nih.gov/pub/OCG-DCC/CTD2/Broad/CTRPv2.2_2015_pub_CancerDisc_5_1210/ 
2. Click "CTRPv2.2_2015_pub_CancerDisc_5_1210.zip" to download the CTRP dataset
3. Move the downloaded "CTRPv2.2_2015_pub_CancerDisc_5_1210.zip" file to the onco-gps-analysis-paper/data directory
4. Run the cell below to prepare the CTRP dataset for analysis.

In [ ]:
unzip('../data/CTRPv2.2_2015_pub_CancerDisc_5_1210.zip')
unzip('../data/gene_set__gene_set_x_ccle_cellline.gct.zip')

# Read compound data
auc = pd.read_table('../data/v22.data.auc_sensitivities.txt')
print(auc.shape)

cpd = pd.read_table('../data/v22.meta.per_compound.txt', index_col=0)
print(cpd.shape)

ccl = pd.read_table('../data/v22.meta.per_cell_line.txt', index_col=0)
print(ccl.shape)

# Make dict for faster ID-to-name look up
cpd_d = cpd['cpd_name'].to_dict()
ccl_d = ccl['ccl_name'].to_dict()

# Make empty compound-x-cellline matrix
compound_x_cellline = pd.DataFrame(
    index=sorted(set(cpd['cpd_name'])), columns=sorted(set(ccl['ccl_name'])))
print(compound_x_cellline.shape)

# Populate compound-x-cellline matrix
for i, (i_cpd, i_ccl, a) in auc.iterrows():

    # Get compound name
    cpd_n = cpd_d[i_cpd]

    # Get cellline name
    ccl_n = ccl_d[i_ccl]

    # Get current AUC
    a_ = compound_x_cellline.loc[cpd_n, ccl_n]

    # If the current AUC is not set, set with this AUC
    if pd.isnull(a_):
        compound_x_cellline.loc[cpd_n, ccl_n] = a

    # If this AUC is smaller than the current AUC, set with this AUC
    elif a < a_:

        print('Updating AUC of compound {} on cellline {}: {:.3f} ==> {:.3f}'.
              format(cpd_n, ccl_n, a_, a))

        compound_x_cellline.loc[cpd_n, ccl_n] = a

# Update cellline names to match CCLE cellline names
columns = list(compound_x_cellline.columns)

# Read CCLE cellline annotations
a = pd.read_table('../data/CCLE_sample_info_file_2012-10-18.txt', index_col=0)

# Get CCLE cellline names
for i, ccl_n in enumerate(compound_x_cellline.columns):

    matches = []

    for ccle_n in a.index:
        if ccl_n.lower() == ccle_n.lower().split('_')[0]:
            matches.append(ccle_n)

    if 0 == len(matches):
        print('0 match: {}; matching substring ...'.format(ccl_n))

        for ccle_n in a.index:

            if ccl_n.lower() in ccle_n.lower():

                print('\t{} ==> {}.'.format(ccl_n, ccle_n))
                matches.append(ccle_n)

    if 1 == len(matches):

        print('{} ==> {}.'.format(ccl_n, matches[0]))
        columns[i] = matches[0]

    else:
        print('1 < matches: {} ==> {}'.format(ccl_n, matches))

# Update with CCLE cellline names
compound_x_cellline.columns = columns

# Write .gct file
ccal.write_gct(compound_x_cellline,
              '../data/ctd2__compound_x_ccle_cellline.gct')

compound_x_cellline

### 5. Check that the datasets are ready for analysis

Run the cell below. If there's no error, you have all the dataset you need and you can continue to Notebook 2 (2 Generating Oncogenic Activation Signature). If there's an error, add the file the error tells you you're missing and run the cell below again.

In [48]:
for fn in [
        'kras_isogenic_and_imortalized_celllines.gct',
        'achilles__gene_x_ccle_cellline.gct',
        'mutation__gene_x_ccle_cellline.gct',
        'rpkm__gene_x_ccle_cellline.gct',
        'gene_set__gene_set_x_ccle_cellline.gct',
        'regulator__gene_set_x_ccle_cellline.gct',
        'rppa__protein_x_ccle_cellline.gct',
        'ctd2__compound_x_ccle_cellline.gct',
        'annotation__feature_x_ccle_cellline.gct',
]:
    assert fn in listdir('../data/'), 'Missing {}!'.format(fn)